<p style="font-family: Open Sans; color: #2B6264; font-weight: bold; font-size: 125%">What are we going to cover?</p>

<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">How can we populate a Perpetual Property with a value for a given entity e.g. Instrument using the LUSID APIs via the LUSID website?</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">How can we decorate the value of a Property onto an entity using the LUSID APIs via the LUSID website?</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">How does the value of a Perpetual Property change throughout its life?</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">How can we populate a Perpetual Property with a value using the LUSID APIs via the Python SDK?</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">How can we decorate Properties onto an entity using the LUSID APIs via the Python SDK?</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">What are all the different shapes that Property Values can take when using the LUSID APIs via the Python SDK?</p></li>
</ul>

<p style="font-family: Open Sans; color: #2B6264; font-weight: bold; font-size: 125%">Populating Perpetual Property Values</p>

In [ ]:
import pprint
import os
import lusid
from lusidjam import RefreshingToken
from finbourne_sdk_utils.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusid.extensions import (
    SyncApiClientFactory,
    ArgsConfigurationLoader,
    EnvironmentVariablesConfigurationLoader,
    SecretsFileConfigurationLoader
)
from lusid.models import *

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")
# Run the Notebook locally using a secrets file (see https://support.lusid.com/docs/how-do-i-use-an-api-access-token-with-the-lusid-sdk)
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
config_loaders=[
    ArgsConfigurationLoader(access_token = RefreshingToken(), app_name = "LusidJupyterNotebook"),
    EnvironmentVariablesConfigurationLoader(),
    SecretsFileConfigurationLoader(secrets_path)]
api_factory = SyncApiClientFactory(config_loaders=config_loaders)
 
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
display(instruments_api)

In [ ]:
instruments_api.upsert_instruments(
    scope="FinbourneUniversity",
    request_body={
        "instr_1": lusid.InstrumentDefinition(
            name="MICROSOFT CORP",
            identifiers={
                "Figi": lusid.InstrumentIdValue(
                    value="BBG000BPH459"
                )
            }
        )
    }
)

In [4]:
property_definitions_api = api_factory.build(lusid.api.PropertyDefinitionsApi)

try:
    property_definitions_api.delete_property_definition(
        domain="Instrument",
        scope="FinbourneUniversity",
        code="Issuer")
except:
    pass

issuer_prop_definition = property_definitions_api.create_property_definition(
    create_property_definition_request=lusid.CreatePropertyDefinitionRequest(
        domain="Instrument",
        scope="FinbourneUniversity",
        code="Issuer",
        display_name="Issuer",
        property_description="The issuer of the security",
        data_type_id=lusid.ResourceId(
            scope="system",
            code="string"
        ),
        life_time="Perpetual",
        constraint_style="Property"
    )
)

In [ ]:
result = instruments_api.upsert_instruments_properties(
    scope="FinbourneUniversity",
    upsert_instrument_property_request=[
        lusid.UpsertInstrumentPropertyRequest(
            identifier_type="Figi",
            identifier="BBG000BPH459",
            properties=[
                lusid.ModelProperty(
                    key="Instrument/FinbourneUniversity/Issuer",
                    value=lusid.PropertyValue(
                        label_value="Microsoft Corporation LTD"
                    )
                )
            ]
        )
    ]
)

print(result)

<p style="font-family: Open Sans; color: #2B6264; font-weight: bold; font-size: 125%">Decorating Property Values</p>

In [ ]:
instrument = instruments_api.get_instrument(
    scope="FinbourneUniversity",
    identifier_type="Figi",
    identifier="BBG000BPH459"
)

print(instrument)


In [ ]:
instrument = instruments_api.get_instrument(
    scope="FinbourneUniversity",
    identifier_type="Figi",
    identifier="BBG000BPH459",
    property_keys=[
        "Instrument/FinbourneUniversity/Issuer"
    ],
)

print(instrument)

In [ ]:
instruments_api.get_instrument(
    scope="FinbourneUniversity",
    identifier_type="Figi",
    identifier="BBG000BPH459",
    property_keys=[
        "Instrument/FinbourneUniversity/Issuer"
    ]
)

In [ ]:
try:
    instrument = instruments_api.get_instrument(
        scope="FinbourneUniversity",
        identifier_type="Figi",
        identifier="BBG000BPH459",
        property_keys=[
            "Instrument/FinbourneUniversity/Issuer"
        ],
        as_at="2023-03-20T04:05:00Z"
    )
except lusid.ApiException as e:
    print(e)

In [ ]:
instrument = instruments_api.get_instrument(
    scope="FinbourneUniversity",
    identifier_type="Figi",
    identifier="BBG000BPH459",
    property_keys=[
        "Instrument/FinbourneUniversity/Issuer"
    ],
    as_at=result.as_at_date,
    effective_at="1912-03-20T04:05:00Z"
)

print(instrument.properties)

<p style="font-family: Open Sans; color: #2B6264; font-weight: bold; font-size: 125%">Populating Different Types of Properties</p>

In [12]:
try:
    property_definitions_api.delete_property_definition(
        domain="Instrument",
        scope="FinbourneUniversity",
        code="IPOPrice")
except:
    pass

issuer_prop_definition = property_definitions_api.create_property_definition(
    create_property_definition_request=lusid.CreatePropertyDefinitionRequest(
        domain="Instrument",
        scope="FinbourneUniversity",
        code="IPOPrice",
        display_name="Initial Public Offering Price",
        property_description="The price of the initial public offering which led to the float of this security",
        data_type_id=lusid.ResourceId(
            scope="system",
            code="iso4217CurrencyAndAmount"
        ),
        life_time="Perpetual",
        constraint_style="Property"
    )
)

In [13]:
try:
    property_definitions_api.delete_property_definition(
        domain="Instrument",
        scope="FinbourneUniversity",
        code="EarningReleases")
except:
    pass

issuer_prop_definition = property_definitions_api.create_property_definition(
    create_property_definition_request=lusid.CreatePropertyDefinitionRequest(
        domain="Instrument",
        scope="FinbourneUniversity",
        code="EarningReleases",
        display_name="Earning Releases",
        property_description="Links to relevant earning releases which affect this security",
        data_type_id=lusid.ResourceId(
            scope="system",
            code="string"
        ),
        life_time="Perpetual",
        constraint_style="Collection"
    )
)

In [ ]:
result = instruments_api.upsert_instruments_properties(
    scope="FinbourneUniversity",
    upsert_instrument_property_request=[
        lusid.UpsertInstrumentPropertyRequest(
            identifier_type="Figi",
            identifier="BBG000BPH459",
            properties=[
                lusid.ModelProperty(
                    key="Instrument/FinbourneUniversity/Issuer",
                    value=lusid.PropertyValue(
                        label_value="Microsoft Corporation LTD"
                    )
                ),
                lusid.ModelProperty(
                    key="Instrument/FinbourneUniversity/IPOPrice",
                    value=lusid.PropertyValue(
                        metric_value=lusid.MetricValue(
                            value=21.00,
                            unit="USD"
                        )
                    )
                ),
                lusid.ModelProperty(
                    key="Instrument/FinbourneUniversity/EarningReleases",
                    value=lusid.PropertyValue(
                        label_value_set=lusid.LabelValueSet(
                            values=[
                            "https://www.microsoft.com/en-us/investor/earnings/fy-2023-q2/press-release-webcast",
                            "https://www.microsoft.com/en-us/investor/earnings/fy-2023-q1/press-release-webcast"
                        ])
                    )
                )
            ]
        )
    ]
)

print(result)

In [ ]:
instrument = instruments_api.get_instrument(
    scope="FinbourneUniversity",
    identifier_type="Figi",
    identifier="BBG000BPH459",
    property_keys=[
        "Instrument/FinbourneUniversity/Issuer",
        "Instrument/FinbourneUniversity/IPOPrice",
        "Instrument/FinbourneUniversity/EarningReleases"
    ]
)

print(instrument.properties)

In [ ]:
property_definitions_api.delete_property_definition(
    domain="Instrument",
    scope="FinbourneUniversity",
    code="Issuer")

property_definitions_api.delete_property_definition(
    domain="Instrument",
    scope="FinbourneUniversity",
    code="IPOPrice")

property_definitions_api.delete_property_definition(
    domain="Instrument",
    scope="FinbourneUniversity",
    code="EarningReleases")

<p style="font-family: Open Sans; color: #2B6264; font-weight: bold; font-size: 125%">What have we covered?</p>

<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">We populated the Microsoft Equity Instrument with a value for the Perpetual "Issuer" Property using the LUSID APIs via the LUSID website.</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">We updated the view in the LUSID website to decorate our "Issuer" Property onto the Instrument.</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">We discussed that although the pattern for working with Properties is broadly the same across entities, the APIs are specific to each entity type i.e. Instruments, Portfolios, Legal Entities etc. and may have some differences in behaviour.</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">We updated the value of our Perpetual "Issuer" Property using the LUSID Upsert Instruments Properties API via the Python SDK.</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">We decorated our Property value onto our Instrument using the LUSID Get Instrument API by specifying the Property Key for our Perpetual "Issuer" Property.</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">We explored how the value of a Perpetual Property changes based on the provided asAt and effectiveAt date times. We saw that it only varies with respect to the asAt date time.</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">We looked at all the possible shapes of a Property value including a Label Value, Metric Value and Label Value Set. We understood how we can determine which shape a Property value will take based on the Constraint Style and the Type of the Property Definition.</p></li>
</ul>